In [1]:
from utils import load_dataset, fill_mask, split_dataset, bool_ext

/home/ksuga/.pyenv/versions/3.7.0/envs/attention/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [256]:
dataset,ptw_ids = load_dataset(input_dir = "data/input", repository = "gdsc", drug_id = -1)

In [297]:
max(ptw_ids)

24

In [7]:
train_set,test_set = split_dataset(datset,ratio=0.8)

In [10]:
train_set["tgt"],train_set['msk'] = fill_mask(train_set['tgt'],train_set['msk'])

In [293]:
train_set['msk'].shape

(676, 260)

In [294]:
a=np.zeros((3,4))
b=np.ones((4,5))
c=[a,b]

In [296]:
c[0][0]

array([0., 0., 0., 0.])

In [283]:
train_set['msk'].type

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [14]:
def load_dataset(input_dir="data/input", repository="gdsc", drug_id=-1, shuffle_feature=False):
  """ Load dataset. Samples will be shuffled and all omics data and sensitivity
  data will be in the same order of samples.

  omics_data: dict
    exp_bin, mut_bin, cnv_bin, met_bin, exp_idx, mut_idx, cnv_idx, met_idx
    tmr, tgt, msk

  """

  assert repository in ['gdsc', 'ccle']

  # load sensitivity data and multi-omics data
  tgt = pd.read_csv(os.path.join(input_dir,repository+'.csv'), index_col=0)

  drug_info = pd.read_csv(os.path.join(input_dir,'drug_info_'+repository+'.csv'), index_col=0)

  ptw_ids = get_ptw_ids(drug_info, tgt, repository)


  omics_data = {'mut':None, 'cnv':None, 'exp':None, 'met':None}
  for omic in omics_data.keys():
    omics_data[omic] = pd.read_csv(
        os.path.join(input_dir,omic+'_'+repository+'.csv'), index_col=0)

  # find samples that have all four types of omics data
  # 846 samples for gdsc, 409 samples for ccle
  common_samples = [v.index for v in omics_data.values()]
  common_samples = list( set(tgt.index).intersection(*common_samples) )

  tgt = tgt.loc[common_samples]
  for omic in omics_data.keys():
    omics_data[omic] = omics_data[omic].loc[common_samples]

  tmr = list(tgt.index) # barcodes/names of tumors
  msk = tgt.notnull().astype(int).values # mask of target data: 1->data available, 0->nan
  tgt = tgt.fillna(0).astype(int).values # fill nan element of target with 0.

  num_sample = len(tmr)

  rng = []
  with open('data/input/rng.txt', 'r') as f:
    for line in f:
      v = int(line.strip())
      if v < num_sample:
        rng.append(v)

  tmr = [tmr[i] for i in rng]
  msk = msk[rng]
  tgt = tgt[rng]

  omics_data_keys = list(omics_data.keys())
  for omic in omics_data_keys:
    omic_val = omics_data.pop(omic)
    omic_val = omic_val.values
    if shuffle_feature:
      # shuffle features of each sample (in place)
      for l in omic_val:
        np.random.shuffle(l)
    omics_data[omic+'_bin'] = omic_val
    omics_data[omic+'_bin'] = omics_data[omic+'_bin'][rng]
    omics_data[omic+'_idx'] = bin2idx(omics_data[omic+'_bin'])

  omics_data['tgt'] = tgt
  omics_data['msk'] = msk
  omics_data['tmr'] = tmr

  if drug_id != -1:
    omics_data["tgt"] = np.expand_dims(tgt[:,drug_id], axis=1)
    omics_data["msk"] = np.expand_dims(msk[:,drug_id], axis=1)

  return omics_data, ptw_ids



In [21]:
import pandas as pd
import os 
input_dir = 'data/input'
tgt = pd.read_csv(os.path.join(input_dir,'gdsc.csv'), index_col=0)


In [48]:
repository ='gdsc'
omics_data = {'mut':None, 'cnv':None, 'exp':None, 'met':None}
for omic in omics_data.keys():
    omics_data[omic] = pd.read_csv(
        os.path.join(input_dir,omic+'_'+repository+'.csv'), index_col=0)


In [49]:
common_samples = [v.index for v in omics_data.values()]

In [53]:
common_samples = list(set(tgt.index).intersection(*common_samples))
print(len(common_samples))
tgt1 = tgt.loc[common_samples]

846


In [54]:
tgt1

,1026,1028,1029,1030,1031,1032,1033,1036,1037,1038,...,87,88,89,94,104,106,110,111,119,127
COSMIC.687514,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COSMIC.906795,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COSMIC.905982,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
COSMIC.910936,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COSMIC.906852,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COSMIC.949161,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COSMIC.949176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COSMIC.949171,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
COSMIC.906808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [71]:
omic_exp = pd.read_csv(os.path.join(input_dir,'exp_gdsc.csv'),index_col=0)
sample_names = [omic_exp.index]

In [76]:
sample_names = list(set(tgt.index).intersection(*sample_names))

In [270]:
if len(sample_names) == len(dataset['tmr']:
    print('yes!')
else:
    print('no!')

yes!


In [77]:
tgt2 = tgt.loc[sample_names]

In [78]:
tgt2

,1026,1028,1029,1030,1031,1032,1033,1036,1037,1038,...,87,88,89,94,104,106,110,111,119,127
COSMIC.687514,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COSMIC.906795,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COSMIC.905982,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
COSMIC.910936,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COSMIC.906852,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COSMIC.949161,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COSMIC.949176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COSMIC.949171,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
COSMIC.906808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [83]:
tgt2.columns

Index(['1026', '1028', '1029', '1030', '1031', '1032', '1033', '1036', '1037',
       '1038',
       ...
       '87', '88', '89', '94', '104', '106', '110', '111', '119', '127'],
      dtype='object', length=260)

In [85]:
tmr = list(tgt2.index)
msk = tgt2.notnull().astype(int).values
tgt = tgt2.fillna(0).astype(int).values
num_sample = len(tmr)

In [87]:
rng = []
with open('data/input/rng.txt', 'r') as f:
    for line in f:
      v = int(line.strip())
      if v < num_sample:
        rng.append(v)

In [105]:
def bin2idx(omic_bin):
  """ Transfer a binarized matrix into a index matrix (for input of embedding layer).

  omic_bin: (num_sample, num_feature), each value in {0,1}
  omic_idx: 0 is used for padding, and therefore meaningful index starts from 1.

  """

  num_max_omic = omic_bin.sum(axis=1).max() # max num of mutation in a single sample
  omic_idx = np.zeros((len(omic_bin), num_max_omic), dtype=int )
  for idx, line in enumerate(omic_bin):
    line = [idy+1 for idy, val in enumerate(line) if val == 1]
    omic_idx[idx][0:len(line)] = line

  return omic_idx

In [106]:
def get_ptw_ids(drug_info, tgt, repository):

  id2pw = {id:pw for id,pw in zip(drug_info.index,drug_info['Target pathway'])}

  if repository == 'gdsc':
    #GDSC
    pws = [id2pw.get(int(c),'Unknown') for c in tgt.columns]
  else:
    #CCLE
    pws = [id2pw.get(c,'Unknown') for c in tgt.columns]

  pw2id = {pw:id for id,pw in enumerate(list(set(pws)))}

  ptw_ids = [pw2id[pw] for pw in pws]

  return ptw_ids

In [107]:
def load_dataset(input_dir="data/input", repository="gdsc", drug_id=-1, shuffle_feature=False):
  """ Load dataset. Samples will be shuffled and all omics data and sensitivity
  data will be in the same order of samples.

  omics_data: dict
    exp_bin, mut_bin, cnv_bin, met_bin, exp_idx, mut_idx, cnv_idx, met_idx
    tmr, tgt, msk

  """

  assert repository in ['gdsc', 'ccle']

  # load sensitivity data and multi-omics data
  tgt = pd.read_csv(os.path.join(input_dir,repository+'.csv'), index_col=0)

  drug_info = pd.read_csv(os.path.join(input_dir,'drug_info_'+repository+'.csv'), index_col=0)

  ptw_ids = get_ptw_ids(drug_info, tgt, repository)


  omics_data = {'mut':None, 'cnv':None, 'exp':None, 'met':None}
  for omic in omics_data.keys():
    omics_data[omic] = pd.read_csv(
        os.path.join(input_dir,omic+'_'+repository+'.csv'), index_col=0)

  # find samples that have all four types of omics data
  # 846 samples for gdsc, 409 samples for ccle
  common_samples = [v.index for v in omics_data.values()]
  common_samples = list( set(tgt.index).intersection(*common_samples) )

  tgt = tgt.loc[common_samples]
  for omic in omics_data.keys():
    omics_data[omic] = omics_data[omic].loc[common_samples]

  tmr = list(tgt.index) # barcodes/names of tumors
  msk = tgt.notnull().astype(int).values # mask of target data: 1->data available, 0->nan
  tgt = tgt.fillna(0).astype(int).values # fill nan element of target with 0.

  num_sample = len(tmr)

  rng = []
  with open('data/input/rng.txt', 'r') as f:
    for line in f:
      v = int(line.strip())
      if v < num_sample:
        rng.append(v)

  tmr = [tmr[i] for i in rng]
  msk = msk[rng]
  tgt = tgt[rng]

  omics_data_keys = list(omics_data.keys())
  for omic in omics_data_keys:
    omic_val = omics_data.pop(omic)
    omic_val = omic_val.values
    if shuffle_feature:
      # shuffle features of each sample (in place)
      for l in omic_val:
        np.random.shuffle(l)
    omics_data[omic+'_bin'] = omic_val
    omics_data[omic+'_bin'] = omics_data[omic+'_bin'][rng]
    omics_data[omic+'_idx'] = bin2idx(omics_data[omic+'_bin'])

  omics_data['tgt'] = tgt
  omics_data['msk'] = msk
  omics_data['tmr'] = tmr

  if drug_id != -1:
    omics_data["tgt"] = np.expand_dims(tgt[:,drug_id], axis=1)
    omics_data["msk"] = np.expand_dims(msk[:,drug_id], axis=1)

  return omics_data, ptw_ids

In [108]:
omics_data,ptw_ids = load_dataset()

In [118]:
omics_data.keys()

dict_keys(['mut_bin', 'mut_idx', 'cnv_bin', 'cnv_idx', 'exp_bin', 'exp_idx', 'met_bin', 'met_idx', 'tgt', 'msk', 'tmr'])

In [120]:
train_set, test_set = split_dataset(omics_data, ratio=0.8)

In [130]:
train_set['tgt'], train_set['msk'] = fill_mask(train_set['tgt'], train_set['msk'])

In [133]:
exp_size = omics_data['exp_bin'].shape[1]
mut_size = omics_data['mut_bin'].shape[1]
cnv_size = omics_data['cnv_bin'].shape[1]

In [245]:
exp_size

3000

In [216]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
class ExpEncoder(nn.Module):
  """ Encoder module with/without self-attention function to encode omic information.

  """

  def __init__(
      self, omc_size, hidden_dim, dropout_rate=0.5, embedding_dim=512,
      use_attention=True, attention_size=400, attention_head=8, init_gene_emb=True,
      use_cntx_attn=True, ptw_ids=None, use_hid_lyr=False, use_relu=False,
      repository='gdsc'):

    """
    Parameters
    ----------
    omc_size: number of input genes whose embeddings will be trained.
    hidden_dim: output hidden layer dimension.
    dropout_rate: dropout rate after each hidden layer.
    embedding_dim: embedding dimentions of genes.
    use_attention: whether use attention mechanism or not.
    attention_size: dimension of linear-tanh transformed embeddings.
    attention_head: number of heads for self-attention mechanism.

    """

    super(ExpEncoder, self).__init__()

    self.use_hid_lyr = use_hid_lyr
    self.use_relu = use_relu
    self.repository = repository

    if init_gene_emb:
      if self.repository == 'gdsc':
        #GDSC dataset
        gene_emb_pretrain = np.genfromtxt('data/input/exp_emb_gdsc.csv', delimiter=',')
      else:
        gene_emb_pretrain = np.genfromtxt('data/input/exp_emb_ccle.csv', delimiter=',')

      self.layer_emb = nn.Embedding.from_pretrained(
          torch.FloatTensor(gene_emb_pretrain), freeze=True, padding_idx=0)

    else:
      self.layer_emb = nn.Embedding(
          num_embeddings=omc_size+1,
          embedding_dim=embedding_dim,
          padding_idx=0)

    self.layer_dropout_0 = nn.Dropout(p=dropout_rate)

    if self.use_hid_lyr:
      self.layer_w_1 = nn.Linear(
          in_features=embedding_dim,
          out_features=hidden_dim,
          bias=True)

      self.layer_dropout_1 = nn.Dropout(p=dropout_rate)

    self.use_attention = use_attention
    self.use_cntx_attn = use_cntx_attn
    # additional self-attention is used if specified
    if self.use_attention:

      self.layer_w_0 = nn.Linear(
          in_features=embedding_dim,
          out_features=attention_size,
          bias=True)

      self.layer_beta = nn.Linear(
          in_features=attention_size,
          out_features=attention_head,
          bias=True)

      if self.use_cntx_attn:
        self.layer_emb_ptw = nn.Embedding(
            num_embeddings=max(ptw_ids)+1,
            embedding_dim=attention_size)


  def forward(self, omc_idx, ptw_ids):
    """
    Parameters
    ----------
    omc_idx: int array with shape (batch_size, num_omc)
      indices of perturbed genes in the omic data of samples

    Returns
    -------

    """

    E_t = self.layer_emb(omc_idx) #(batch_size, num_omc, embedding_dim)


    if self.use_attention:

      E_t = torch.unsqueeze(E_t,1) #(batch_size, 1, num_omc, embedding_dim)
      E_t = E_t.repeat(1,len(ptw_ids),1,1) #(batch_size, num_drg, num_omc, embedding_dim)

      if self.use_cntx_attn:
        Ep_t = self.layer_emb_ptw(ptw_ids) #(1, num_drg, attention_size)
        Ep_t = torch.unsqueeze(Ep_t,2) #(1, num_drg, 1, attention_size)
        Ep_t = Ep_t.repeat(omc_idx.shape[0],1,omc_idx.shape[1],1) #(batch_size, num_drg, num_omc, attention_size)

        E_t_1 = torch.tanh( self.layer_w_0(E_t) + Ep_t) #(batch_size, num_drg, num_omc, attention_size)

      else:
        E_t_1 = torch.tanh( self.layer_w_0(E_t) ) #(batch_size, num_omc, attention_size)

      A_omc = self.layer_beta(E_t_1) #(batch_size, num_drg, num_omc, attention_head)

      A_omc = F.softmax(A_omc, dim=2) #(batch_size, num_drg, num_omc, attention_head)
      A_omc = torch.sum(A_omc, dim=3, keepdim=True) #(batch_size, num_drg, num_omc, 1)

      #(batch_size, num_drg, 1, num_omc) * (batch_size, num_drg, num_omc, embedding_dim)
      #=(batch_size, num_drg, 1, embedding_dim)

      self.Amtr = torch.squeeze(A_omc, 3) #(batch_size, num_drg, num_omc)

      emb_omc = torch.sum( torch.matmul(A_omc.permute(0,1,3,2), E_t), dim=2, keepdim=False) #(batch_size, num_drg, embedding_dim)
      print('emb_omc',emb_omc)

    else:

      emb_omc = torch.mean(E_t, dim=1, keepdim=False) #(batch_size, embedding_dim)

      emb_omc = torch.unsqueeze(emb_omc,1) #(batch_size, 1, embedding_dim)

      emb_omc = emb_omc.repeat(1,len(ptw_ids),1) #(batch_size, num_drg, embedding_dim)
      


    if self.use_relu:
      hid_omc = self.layer_dropout_0(torch.relu(emb_omc))
    else:
      hid_omc = self.layer_dropout_0(emb_omc)

    return hid_omc


In [217]:
encoder = ExpEncoder(
        omc_size=exp_size, hidden_dim=200,dropout_rate=0.5 ,
        embedding_dim=200, use_attention=True,
        attention_size=128, attention_head=8,
        init_gene_emb=True, use_cntx_attn=True, ptw_ids=ptw_ids,
        use_hid_lyr=False, use_relu=False, repository='gdsc')


In [218]:
def wrap_dataset_cuda(dataset, use_cuda):
  """ Wrap default numpy or list data into PyTorch variables.
  """

  batch_dataset = {'tmr':dataset['tmr']}
  for k in ['tgt', 'msk']:
    if k in dataset.keys():
      batch_dataset[k] = torch.FloatTensor(dataset[k])

  for k in dataset.keys():
    if k.endswith('_idx'):
      batch_dataset[k] = torch.LongTensor(dataset[k])
    elif k.endswith('_bin'):
      batch_dataset[k] = torch.FloatTensor(dataset[k])

  if use_cuda:
    for k in batch_dataset.keys():
      if k == 'tmr':
        continue
      else:
        batch_dataset[k] = batch_dataset[k].cuda()

  return batch_dataset


In [219]:
def get_minibatch(dataset, rng, index, batch_size, batch_type="train", use_cuda=True):
  """ Get a mini-batch dataset for training or test -- Multi-task/label
  learning version here, so we can take drug reponses of a cell lines as
  a single sample.

  Parameters
  ----------
  dataset: dict
    dict of lists, including SGAs, cancer types, DEGs, patient barcodes
  rng: list of id_tmr
  index: int
    starting index of current mini-batch
  batch_size: int
  batch_type: str
    batch strategy is slightly different for training and test
    "train": will return to beginning of the queue when `index` out of range
    "test": will not return to beginning of the queue when `index` out of range

  Returns
  -------
  batch_dataset: dict
    a mini-batch of the input `dataset`.

  """

  size_rng = len(rng)
  if batch_type == "train":
    batch_dataset = {
        k : [ dataset[k][rng[i%size_rng]] for i in range(index, index+batch_size) ] \
        for k in dataset.keys()}
  elif batch_type == "test":
    batch_dataset = {
        k : [ dataset[k][rng[i]] for i in range(index, min(index+batch_size, size_rng)) ] \
        for k in dataset.keys()}

  batch_dataset = wrap_dataset_cuda(batch_dataset, use_cuda)

  return batch_dataset

In [220]:
batch_set = get_minibatch(train_set, rng, index=0,batch_size=4,batch_type='train',use_cuda=False)

In [255]:
batch_set.keys()

dict_keys(['tmr', 'tgt', 'msk', 'mut_bin', 'mut_idx', 'cnv_bin', 'cnv_idx', 'exp_bin', 'exp_idx', 'met_bin', 'met_idx'])

In [254]:
omc_idx = batch_set['exp_idx']
print(omc_idx.shape)

torch.Size([4, 1500])


In [241]:
ptw_ids = 0

<function Tensor.long>

AttributeError: 'builtin_function_or_method' object has no attribute 'shape'

In [238]:
hid_omc = encoder(omc_idx,ptw_ids)

TypeError: object of type 'builtin_function_or_method' has no len()

In [179]:
len(ptw_ids)

260

In [212]:
batch_dataset['tgt'] = torch.FloatTensor(dataset['tgt'])

In [214]:
batch_dataset['tgt'].shape

torch.Size([846, 260])

In [246]:
class Encoder(nn.Module):
  """ Encoder module with/without self-attention function to encode omic information.
      Expencoder -> Encoder 
  """

  def __init__(
      self, omc_size, hidden_dim, dropout_rate=0.5, embedding_dim=512,
      use_attention=True, attention_size=400, attention_head=8, init_gene_emb=True,
      use_cntx_attn=True, ptw_ids=None, use_hid_lyr=False, use_relu=False,
      repository='gdsc'):

    """
    Parameters
    ----------
    omc_size: number of input genes whose embeddings will be trained.
    hidden_dim: output hidden layer dimension.
    dropout_rate: dropout rate after each hidden layer.
    embedding_dim: embedding dimentions of genes.
    attention_size: dimension of linear-tanh transformed embeddings.
    attention_head: number of heads for self-attention mechanism.

    """

    super(Encoder, self).__init__()
    
    gene_emb_pretrain = np.genfromtxt('data/input/exp_emb_gdsc.csv', delimiter=',')
     

    self.layer_emb = nn.Embedding.from_pretrained(torch.FloatTensor(gene_emb_pretrain), freeze=True, padding_idx=0)

    self.layer_dropout_0 = nn.Dropout(p=dropout_rate)
 
    self.layer_w_0 = nn.Linear(in_features=embedding_dim,out_features=attention_size,bias=True)

    self.layer_beta = nn.Linear(in_features=attention_size,out_features=attention_head,bias=True)
      
    self.layer_emb_ptw = nn.Embedding(num_embeddings=max(ptw_ids)+1,embedding_dim=attention_size)


  def forward(self, omc_idx, ptw_ids):
    """
    Parameters
    ----------
    omc_idx: int array with shape (batch_size, num_omc)
      indices of perturbed genes in the omic data of samples

    Returns
    -------

    """

    E_t = self.layer_emb(omc_idx) #(batch_size, num_omc, embedding_dim)
    E_t = torch.unsqueeze(E_t,1) #(batch_size, 1, num_omc, embedding_dim)
    E_t = E_t.repeat(1,ptw_ids.shape[1],1,1) #(batch_size, num_drg, num_omc, embedding_dim)


    Ep_t = self.layer_emb_ptw(ptw_ids) #(1, num_drg, attention_size)
    Ep_t = torch.unsqueeze(Ep_t,2) #(1, num_drg, 1, attention_size)
    Ep_t = Ep_t.repeat(omc_idx.shape[0],1,omc_idx.shape[1],1) #(batch_size, num_drg, num_omc, attention_size)

    E_t_1 = torch.tanh( self.layer_w_0(E_t) + Ep_t) #(batch_size, num_drg, num_omc, attention_size)



    A_omc = self.layer_beta(E_t_1) #(batch_size, num_drg, num_omc, attention_head)

    A_omc = F.softmax(A_omc, dim=2) #(batch_size, num_drg, num_omc, attention_head)
    A_omc = torch.sum(A_omc, dim=3, keepdim=True) #(batch_size, num_drg, num_omc, 1)

    #(batch_size, num_drg, 1, num_omc) * (batch_size, num_drg, num_omc, embedding_dim)
    #=(batch_size, num_drg, 1, embedding_dim)

    self.Amtr = torch.squeeze(A_omc, 3) #(batch_size, num_drg, num_omc)

    emb_omc = torch.sum( torch.matmul(A_omc.permute(0,1,3,2), E_t), dim=2, keepdim=False) #(batch_size, num_drg, embedding_dim)

    hid_omc = self.layer_dropout_0(emb_omc)

    return hid_omc


In [248]:
encoder = Encoder(omc_size = exp_size,hidden_dim=200)

TypeError: 'NoneType' object is not iterable

In [323]:
a=np.zeros((3,4))
a_torch = torch.Tensor(a).long()

In [325]:
print(type(a_torch))


<class 'torch.Tensor'>


In [330]:
t = torch.ones(2,3,dtype = torch.long)
print(type(t))

<class 'torch.Tensor'>


In [318]:
msks=batch_set['msk']
type(msks)


torch.Tensor

In [304]:
import torch.nn as nn
ptw_ids=None
ptw_emb = nn.Embedding(num_embeddings=max(ptw_ids)+1,embedding_dim=400)

TypeError: 'NoneType' object is not iterable

AttributeError: 'int' object has no attribute 'type'